### scraping prop lines

In [6]:
# initial run(0) or update run (1)
update = 0
# https://www.rotowire.com/betting/mlb/player-props.php
# if you don't want to restart kernal, dump any existing data
try:
    del data
except:
    pass

import requests, time, re, os, json
#import mysql.connector
#import mysql, pymysql, pyodbc
#import sqlalchemy as sal
#from sqlalchemy import create_engine
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
#from bs4 import BeautifulSoup as bs
from datetime import date

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#importing credentials from txt file
with open('../../../Notes-General/config.txt', 'r') as f:
    creds = f.read()
creds = json.loads(creds)

# path to downloads folder
download_path = creds['downloads']

#sportsdataio MLB API
mlbKey = creds['sportsdataiomlbAPI']

#### using the save as button on the page to save the files individually.
#### the rows of the tables that are not visible are hidden from HTML
#### saving is the only way to get all the rows that I know at this time

# path to webdriver for selenium
try:
    service = Service(r"..\browser\geckodriver.exe")
    driver = webdriver.Firefox(service=service)
    waitTime = 10

    site = "https://www.rotowire.com/betting/mlb/player-props.php"
    driver.get(site)
except:
    driver.close()

data_prop_names = ["moneyline-props","props-strikeouts","props-er",
                   "props-bases","props-runs"]
missing = []
for i in range(len(data_prop_names)):
# find the element in the source
    try:
        ele = driver.find_element(By.CSS_SELECTOR, "div[id={}]".format(data_prop_names[i]))
        # retrieve its page coords
        loc = ele.location
        #scroll to the location to activate the table creation
        driver.execute_script("window.scrollTo(0, {});".format(str(loc["y"])))
        # pause for data loading
        time.sleep(3)
        ###### save to csv 
        export_button = ele.find_element(By.CLASS_NAME, "is-csv")
        export_button.click()
    except Exception as e:
        print(data_prop_names[i], e)
        missing.append(i)
        continue

driver.close()

# columns to use for renaming csv's
columns = ['PLAYER', 'team', 'opp', 'dk_line', 'dk_oOdds', 'dk_uOdds',
          'fd_line', 'fd_oOdds', 'fd_uOdds','m_line', 'm_oOdds', 'm_uOdds',
          'pb_line', 'pb_oOdds', 'pb_uOdds']
# prop names to store
db_names = ["moneyline-props","K","ER","TB","R"]

# will be used to get the correct file number when there are missing files
count = 0
# loop through each file to agg into a single data frame
for i in range(0, len(data_prop_names)):
    #uses the indices for the missing odds files
    if i in missing:
        continue
    else:
        stat = data_prop_names[i]
        if i == 0:
            path = download_path + "mlb-player-props-rotowire.csv"
            raw_data = pd.read_csv(path, skiprows=1)
            
            batter_props = ["H", "Hx2", "HR", "RBI", "SB"]
            
            df_hit = raw_data.iloc[:,[0,1,2, 3,4,5,6]]
            df_hit.columns = ['PLAYER', 'team', 'opp',
                              'dk_oOdds', 'fd_oOdds', 'm_oOdds', 'pb_oOdds']
            df_hit.loc[:,['dk_line', 'fd_line','m_line','pb_line']] = 0.5
            df_hit.loc[:,['dk_uOdds', 'fd_uOdds','m_uOdds', 'pb_uOdds']] = np.nan
            df_hit.loc[:,['stat']] = "H"
            
            df_twoHit = raw_data.iloc[:,[0,1,2, 7,8,9,10]]
            df_twoHit.columns = ['PLAYER', 'team', 'opp',
                              'dk_oOdds', 'fd_oOdds', 'm_oOdds', 'pb_oOdds']
            df_twoHit.loc[:,['dk_line', 'fd_line','m_line','pb_line']] = 1.5
            df_twoHit.loc[:,['dk_uOdds', 'fd_uOdds','m_uOdds', 'pb_uOdds']] = np.nan
            df_twoHit.loc[:,['stat']] = "Hx2"
            
            df_hr = raw_data.iloc[:,[0,1,2, 11,12,13,14]]
            df_hr.columns = ['PLAYER', 'team', 'opp',
                              'dk_oOdds', 'fd_oOdds', 'm_oOdds', 'pb_oOdds']
            df_hr.loc[:,['dk_line', 'fd_line','m_line','pb_line']] = 0.5
            df_hr.loc[:,['dk_uOdds', 'fd_uOdds','m_uOdds', 'pb_uOdds']] = np.nan
            df_hr.loc[:,['stat']] = "HR"
            
            df_rbi = raw_data.iloc[:,[0,1,2, 15,16,17,18]]
            df_rbi.columns = ['PLAYER', 'team', 'opp',
                              'dk_oOdds', 'fd_oOdds', 'm_oOdds', 'pb_oOdds']
            df_rbi.loc[:,['dk_line', 'fd_line','m_line','pb_line']] = 0.5
            df_rbi.loc[:,['dk_uOdds', 'fd_uOdds','m_uOdds', 'pb_uOdds']] = np.nan
            df_rbi.loc[:,['stat']] = "RBI"
            
            df_sb = raw_data.iloc[:,[0,1,2, 19,20,21,22]]
            df_sb.columns = ['PLAYER', 'team', 'opp',
                              'dk_oOdds', 'fd_oOdds', 'm_oOdds', 'pb_oOdds']
            df_sb.loc[:,['dk_line', 'fd_line','m_line','pb_line']] = 0.5
            df_sb.loc[:,['dk_uOdds', 'fd_uOdds','m_uOdds', 'pb_uOdds']] = np.nan
            df_sb.loc[:,['stat']] = "SB"
            
            data = pd.concat([df_hit, df_twoHit, df_hr, df_rbi, df_sb])

               
        elif 0 in missing:
            path = download_path + "mlb-player-props-rotowire.csv"
            data = pd.read_csv(path, skiprows=1)
            data.columns=columns
            data['stat'] = db_names[i]
        else:
            count += 1
            path = download_path + "mlb-player-props-rotowire({}).csv".format(count)
            temp = pd.read_csv(path, skiprows=1)
            temp.columns=columns
            temp['stat'] = db_names[i]
            # concat data into main df    
            data = pd.concat([data, temp])
        #delete file after use
        os.remove(path)
# using dk line as the default and using the min of the other options when DK is not available
data["line"] = np.where(data["dk_line"].isna(), 
                       data[["fd_line","m_line","pb_line"]].min(axis=1, skipna=True),
                       data["dk_line"])
data["oOdds"] = np.where(data["dk_oOdds"].isna(), 
                       data[["fd_oOdds","m_oOdds","pb_oOdds"]].max(axis=1, skipna=True),
                       data["dk_oOdds"])
data["uOdds"] = np.where(data["dk_uOdds"].isna(), 
                       data[["fd_uOdds","m_uOdds","pb_uOdds"]].max(axis=1, skipna=True),
                       data["dk_uOdds"])

data = data[["PLAYER", "team", "stat", "line", "oOdds", "uOdds"]]
data = data.dropna(subset=['oOdds'])
# adding the date of the odds
data.loc[:,'date'] = date.today()  

date_string = str(date.today())
# if this is not an update run then the data will be saved. If it is an updated (1) then
# the next cell will need to be executed to filter
if update == 0:
    data.to_csv("../data/{}_MLBodds.csv".format(date_string), index=False)
else:
    # updating previously pulled odds
    # if teams have completed or started their games they need to be filtered 
    # from the newest pull bec the roto site does not accurately save lines once the game start

    # user input to 
    filter_teams = input("Enter Teams Abbrev to filter separated by space >>").split()

    #filtering the input teams from the updated odds data
    data = data[~data['team'].isin(filter_teams)]
    # setting player as index temporarily 
    data.set_index("PLAYER", inplace=True)

    # import the prior odds data saved
    prevOddsData = pd.read_csv("../data/{}_MLBodds.csv".format(date_string), index_col="PLAYER")

    # keep players data in the old data when they aren't present in the new data and then add
    # all of the new data (this achieves the update portion
    data = pd.concat([data[~data.index.isin(prevOddsData.index)], prevOddsData]).reset_index()

    data.to_csv("../data/{}_MLBodds.csv".format(date_string), index=False)
print("done")

done


In [2]:
## import pybaseball
from pybaseball import statcast
import statsapi
pybaseball.cache.enable()

NameError: name 'pybaseball' is not defined

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
# py baseball
data = statcast(start_dt="2023-03-15", end_dt="2023-04-28")

In [ ]:
df = data.copy()

In [ ]:
df['events'].unique()